In [1]:
"""

啟用伺服器基本樣板

"""

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("./line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "./images/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [2]:
menuRawData="""
{
  "size": {
    "width": 2500,
    "height": 1686
  },
  "selected": true,
  "name": "圖文選單 1",
  "chatBarText": "查看更多資訊",
  "areas": [
    {
      "bounds": {
        "x": 8,
        "y": 4,
        "width": 1234,
        "height": 831
      },
      "action": {
        "type": "message",
        "text": "[::text:]我想更認識你"
      }
    },
    {
      "bounds": {
        "x": 8,
        "y": 852,
        "width": 1234,
        "height": 826
      },
      "action": {
        "type": "message",
        "text": "[::text:]讓我看看你的點點滴滴"
      }
    },
    {
      "bounds": {
        "x": 1263,
        "y": 8,
        "width": 1229,
        "height": 827
      },
      "action": {
        "type": "message",
        "text": "[::text:]校園經歷精彩嗎"
      }
    },
    {
      "bounds": {
        "x": 1263,
        "y": 856,
        "width": 1229,
        "height": 822
      },
      "action": {
        "type": "message",
        "text": "[::text:]太神拉"
      }
    }
  ]
}
"""

In [3]:
import json
secretFileContentJson=json.load(open("./line_secret_key",'r'))

print(secretFileContentJson.get("channel_access_token"))
print(secretFileContentJson.get("secret_key"))
print(secretFileContentJson.get("self_user_id"))

BG+HZ2BI4laibJcTO2kie46l/LulEzfdwaBSQ4mvt81OkOrTA3gj5MUzl5wmkVrVtv6ptROH+JMPfdHHcTrGyOfAcmRukoYa4C/1zMf1Ce8+ejdS9bUwKfUESBS4aVb5DCo8N2+NquHvnkxCKlTwEQdB04t89/1O/w1cDnyilFU=
00335a496a54045ab220318d12e3ad8d
U526e4526e2a46e2f815b080d6fd4ac56


In [4]:
import requests

menuJson=json.loads(menuRawData)

createMenuEndpoint = 'https://api.line.me/v2/bot/richmenu'
createMenuRequestHeader={'Content-Type':'application/json','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

#print(createMenuRequestHeader)

lineCreateMenuResponse = requests.post(createMenuEndpoint,headers=createMenuRequestHeader,data=json.dumps(menuJson))

print(lineCreateMenuResponse)
print(lineCreateMenuResponse.text)

<Response [200]>
{"richMenuId":"richmenu-b8b9fa636b19f7dfff5f1fcbcc5c7218"}


In [5]:
# 取得菜單Id 
uploadRichMenuId=json.loads(lineCreateMenuResponse.text).get("richMenuId")
print(uploadRichMenuId)
#'https://api.line.me/v2/bot/richmenu/{richMenuId}/content'

# 設定Line的遠端位置
uploadMenuEndpoint='https://api.line.me/v2/bot/richmenu/%s/content' % uploadRichMenuId
print(uploadMenuEndpoint)

# 設定消息的基本安全憑證
uploadMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 上傳照片
uploadImageFile=open("./images/richmenu.jpg",'rb')
lineUploadMenuResponse=requests.post(uploadMenuEndpoint,headers=uploadMenuRequestHeader,data=uploadImageFile)

print(lineUploadMenuResponse)
print(lineUploadMenuResponse.text)

richmenu-b8b9fa636b19f7dfff5f1fcbcc5c7218
https://api.line.me/v2/bot/richmenu/richmenu-b8b9fa636b19f7dfff5f1fcbcc5c7218/content
<Response [200]>
{}


In [6]:
# 取得菜單id
linkRichMenuId=json.loads(lineCreateMenuResponse.text).get("richMenuId")

# 將菜單id與用戶id組合成遠端位置
linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (secretFileContentJson["self_user_id"], linkRichMenuId)
print(linkMenuEndpoint)

# 設定消息基本安全憑證
linkMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 發送消息
lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
print(lineLinkMenuResponse)
print(lineLinkMenuResponse.text)

https://api.line.me/v2/bot/user/U526e4526e2a46e2f815b080d6fd4ac56/richmenu/richmenu-b8b9fa636b19f7dfff5f1fcbcc5c7218
<Response [200]>
{}


In [7]:
# 取出用戶id，設定Line的遠端位置
userMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu' % (secretFileContentJson["self_user_id"])
print(userMenuEndpoint)

# 設定消息的基本安全憑證
userMenuRequestHeader={'Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 發送消息告知
lineUserMenuResponse=requests.get(userMenuEndpoint,headers=userMenuRequestHeader)
print(lineUserMenuResponse)
print(lineUserMenuResponse.text)

https://api.line.me/v2/bot/user/U526e4526e2a46e2f815b080d6fd4ac56/richmenu
<Response [200]>
{"richMenuId":"richmenu-b8b9fa636b19f7dfff5f1fcbcc5c7218"}


In [8]:
'''

製作文字與圖片的教學訊息

'''
# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 消息清單
reply_message_list = [
TextSendMessage(text="您好，我是小秘書，在這裡我能幫您介紹煜晉這位好青年"),

]

In [9]:
'''

撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 把先前製作好的自定義菜單，與用戶做綁定
3. 回應用戶，歡迎用的文字消息與圖片消息

'''


# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

# 載入requests套件
import requests


# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
    with open("./users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\r\n')
        
        
    # 將菜單綁定在用戶身上
    linkRichMenuId=secretFileContentJson.get("rich_menu_id")
    linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (event.source.user_id, linkRichMenuId)
    linkMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}
    lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
    
    # 回覆文字消息與圖片消息
    line_bot_api.reply_message(
        event.reply_token,
        reply_message_list
    )

In [10]:
#引入所需要的消息與模板消息
from linebot.models import (
    MessageEvent, TemplateSendMessage , PostbackEvent
)

#引入按鍵模板
from linebot.models.template import(
    ButtonsTemplate, CarouselTemplate
)


'''
alt_text: Line簡覽視窗所出現的說明文字
template: 所使用的模板
ButtonsTemplate: 按鍵模板
    thumbnail_image_url: 展示圖片
    title: 標題
    text: 說明文字
    actions: 模板行為所使用的行為
    data: 觸發postback後用戶回傳值，可以對其做商業邏輯處理

'''

'\nalt_text: Line簡覽視窗所出現的說明文字\ntemplate: 所使用的模板\nButtonsTemplate: 按鍵模板\n    thumbnail_image_url: 展示圖片\n    title: 標題\n    text: 說明文字\n    actions: 模板行為所使用的行為\n    data: 觸發postback後用戶回傳值，可以對其做商業邏輯處理\n\n'

In [11]:
buttons_template_message = TemplateSendMessage(
    alt_text='Buttons template',
    template=ButtonsTemplate(
        title='我想更認識你',
        text='請點擊下方按鈕獲得更多資訊',
        actions=[
          {
            "type": "postback",
            "label": "家庭介紹",
            "text": "[::text:]家庭介紹",
            "data": "Data1"
          },
          {
            "type": "postback",
            "label": "打工經驗",
            "text": "[::text:]打工經驗",
            "data": "Data2"
          },
          {
            "type": "postback",
            "label": "未來目標",
            "text": "[::text:]未來目標",
            "data": "Data3"
          }
        ],
)
)

In [12]:
buttons_template_message2 = TemplateSendMessage(
    alt_text='Buttons template',
    template=ButtonsTemplate(
        title='讓我看看你的點點滴滴',
        text='請點擊下方按鈕到我的Github逛逛',
        actions=[
          {
            "type": "postback",
            "label": "Github網址",
            "text": "[::text:]Github網址",
            "data": "Data1"
          },
          {
            "type": "postback",
            "label": "心得",
            "text": "[::text:]心得",
            "data": "Data2"
          }
        ],
)
)

In [13]:
buttons_template_message3 = TemplateSendMessage(
    alt_text='Buttons template',
    template=ButtonsTemplate(
        title='校園經歷精彩嗎',
        text='請點擊下方按鈕獲得更多資訊',
        actions=[
          {
            "type": "postback",
            "label": "大學",
            "text": "[::text:]大學",
            "data": "Data1"
          },
          {
            "type": "postback",
            "label": "高中",
            "text": "[::text:]高中",
            "data": "Data2"
          }
        ],
)
)

In [14]:
buttons_template_message4 = TemplateSendMessage(
    alt_text='Buttons template',
    template=ButtonsTemplate(
        title='太神拉',
        text='請點擊下方按鈕獲得更多資訊',
        actions=[
          {
            "type": "postback",
            "label": "獎狀",
            "text": "[::text:]獎狀",
            "data": "Data1"
          },
          {
            "type": "postback",
            "label": "成績單",
            "text": "[::text:]成績單",
            "data": "Data2"
          }
        ],
)
)

In [15]:
reply_message_list10 = [
    TextSendMessage(text="您好，我是徐煜晉，今年22歲。我出生於民風純樸的彰化市，在家中排行老么，上面還有哥哥及姊姊。爸爸的職業是冷凍空調技術師，由於爸爸常常將冷氣拆開來修理後再組裝回去，耳濡目染下，我也喜歡了解各部件的功能後組裝成可以順利運作的樣貌，就如同程式設計一樣"),
    ]

In [16]:
reply_message_list2 = [
    TextSendMessage(text="作業員：在高中畢業時，我曾利用暑假時去工廠當產線作業員，在那邊我學到了按部就班及作業流程的重要性，就像串聯的電路一樣，只要其中一環出現差錯，整條產線都會被影響，所以認真做好每個人的工作便是對團隊最基本的負責態度。"),
    TextSendMessage(text="服務生：我曾在餐廳外場擔任服務生，在外場時常會遇到各種臨時狀況，且任務時常要在同時進行，所以在短時間內判斷事情的輕重緩急便是最重要的能力，並且也要能夠靈活地運用各種身邊的資源，與夥伴共同達成目標。"),
    TextSendMessage(text="實習生：在大學時期，我在船廠擔任實習生，除了在室內學習各種設計的技巧，也要親自到工廠裡接觸實作的部分，在不斷接觸設計端及實作端之後，我發現很多事情只是理論上可行，但實際上卻會有很多難處，更確信了團隊溝通的重要性了。")
]

In [17]:
reply_message_list3 = [
    TextSendMessage(text="短期：盡快熟悉python的各項運用，在出錯時可以獨立解決問題，遇到問題時可以快速找到解決問題的方法。"),
    TextSendMessage(text="中期：熟悉更多種語言，了解網路部屬的概念，並且實作有用的工具提供需要的人能利用。"),
    TextSendMessage(text="長期：學習管理及經營，試圖找出自己的特色，並把創意化作產品，成為社會進步的動力。")
]

In [18]:
reply_message_list4 = [
    TextSendMessage(text="https://github.com/TheQ5"),
]

In [19]:
reply_message_list5 = [
    TextSendMessage(text="Coming Soon"),
]

In [20]:
reply_message_list6 = [
    TextSendMessage(text="在大學時期，除了我自己本科系的學科保持在全系前20%以外，我還會利用時間去修資訊相關的課程，如：資料結構與演算法、作業系統等等，我認為將自己的所學與其他領域合併，更能體現它的價值，也是未來必不可少的技能。透過學習更多科目，也讓我更能把知識融會貫通。"),
    TextSendMessage(text="社團方面，我參加系上的籃球隊，透過隊員一起練習籃球基本功，感情逐漸增溫，我也非常享受與團隊一起克服難關後的成就感")
]

In [21]:
reply_message_list7 = [
    TextSendMessage(text="高職時期，我就讀模具科，透過不斷練習手藝，成為科內的選手，參加全國技能競賽，雖然結果並沒有得名，不過這次寶貴的經驗也讓我知道人外有人，並且深刻體會到，想要得到更好的技術，就要付出更多的努力去練習。"),
]

In [22]:
reply_message_list8 = [
#     TextSendMessage(text="服務學習,表現優異(上學期)"),
    ImageSendMessage(original_content_url='https://cdn.discordapp.com/attachments/627815792217554948/627815974682492949/323dafe576214b98.jpg' ,
    preview_image_url='https://cdn.discordapp.com/attachments/627815792217554948/627815974682492949/323dafe576214b98.jpg'),
#     TextSendMessage(text="服務學習,表現優異(下學期)"),
    ImageSendMessage(original_content_url='https://cdn.discordapp.com/attachments/627815792217554948/627815986158108683/2.jpg' ,
    preview_image_url='https://cdn.discordapp.com/attachments/627815792217554948/627815986158108683/2.jpg'),
#     TextSendMessage(text="國文會考特優"),
    ImageSendMessage(original_content_url='https://cdn.discordapp.com/attachments/627815792217554948/627816016327606283/4.jpg' ,
    preview_image_url='https://cdn.discordapp.com/attachments/627815792217554948/627816016327606283/4.jpg'),
#     TextSendMessage(text="操性成績全班第三"),
    ImageSendMessage(original_content_url='https://cdn.discordapp.com/attachments/627815792217554948/627816003824254976/3.jpg' ,
    preview_image_url='https://cdn.discordapp.com/attachments/627815792217554948/627816003824254976/3.jpg')
    ]

In [23]:
reply_message_list9 = [
    TextSendMessage(text="歷年成績單"),
    ImageSendMessage(original_content_url='https://cdn.discordapp.com/attachments/627815792217554948/627816030059626516/001.jpg' ,
    preview_image_url='https://cdn.discordapp.com/attachments/627815792217554948/627816030059626516/001.jpg')
]

In [24]:
template_message_dict = {
    "[::text:]我想更認識你":buttons_template_message,
    "[::text:]讓我看看你的點點滴滴":buttons_template_message2,
    "[::text:]校園經歷精彩嗎":buttons_template_message3,
    "[::text:]太神拉":buttons_template_message4,
    "[::text:]家庭介紹":reply_message_list10,
    "[::text:]打工經驗":reply_message_list2,
    "[::text:]未來目標":reply_message_list3,
    "[::text:]Github網址":reply_message_list4,
    "[::text:]心得":reply_message_list5,
    "[::text:]大學":reply_message_list6,
    "[::text:]高中":reply_message_list7,
    "[::text:]獎狀":reply_message_list8,
    "[::text:]成績單":reply_message_list9
}

In [25]:

# 用戶發出文字消息時， 按條件內容, 回傳照片地圖
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
    )

In [26]:
'''

當用戶發出文字消息時，判斷文字內容是否包含[::text:]，
    若有，則從template_message_dict 內找出相關訊息
    若無，則回傳預設訊息。

'''

# 用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    
    if(event.message.text.find('::text:')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text))
    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="請點擊菜單上圖面，或輸入[::text:]more，取得更多幫助"))

In [27]:
# @handler.add(PostbackEvent)
# def handle_post_message(event):
#     user_profile = line_bot_api.get_profile(event.source.user_id)
#     if (event.postback.data.find('Data1')== 0):
#         with open("../user_profile_business.txt", "a") as myfile:
#             myfile.write(json.dumps(vars(user_profile),sort_keys=True))
#             myfile.write('\r\n')
#             line_bot_api.reply_message(
#             event.reply_token,
#                 TextMessage(
#                     text='請稍待，會有專人與您聯繫'
#                 )
#             )
#     elif (event.postback.data.find('Data2') == 0):
#         with open("../user_profile_tutorial.txt", "a") as myfile:
#             myfile.write(json.dumps(vars(user_profile),sort_keys=True))
#             myfile.write('\r\n')
#             line_bot_api.reply_message(
#             event.reply_token,
#                 TextMessage(
#                     text='請稍待，我們會派專家與您聯繫'
#                 )
#             )
#     else:
#         pass

In [ ]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Sep/2019 23:20:52] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2019 23:20:54] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2019 23:20:58] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2019 23:21:06] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2019 23:21:07] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2019 23:22:05] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2019 23:22:17] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2019 23:22:41] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2019 23:22:42] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2019 23:23:29] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2019 23:23:44] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2019 23:26:59] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2019 23:27:02] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2019 23:27:06] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2019 23:28:04] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/201

127.0.0.1 - - [30/Sep/2019 23:51:09] "POST / HTTP/1.1" 500 -
127.0.0.1 - - [30/Sep/2019 23:52:06] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2019 23:52:07] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2019 23:52:30] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2019 23:52:31] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2019 23:52:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2019 23:52:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2019 23:52:42] "POST / HTTP/1.1" 200 -


In [ ]:
# app.run()